<a href="https://colab.research.google.com/github/TomMaullin/BLMM-sandbox/blob/master/Matrix_Lemmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix reshape lemmas

This notebook contains all matrix reshaping lemmas found during the BLMM project.

## Setup

In [0]:
import numpy as np
import time
import scipy.sparse

### Helper functions

This section contains helper functions used later in the notebook.

In [0]:
def mat2vec(matrix):
  
  #Return vectorised matrix
  return(matrix.transpose().reshape(matrix.shape[0]*matrix.shape[1],1))

def vec2mat(vec): # Only for square matrices
  
  # Return matrix
  return(vec.reshape(np.int64(np.sqrt(vec.shape[0])),np.int64(np.sqrt(vec.shape[0]))).transpose())

# Block vec with row-wise chunks of size n
def matb2vecb(matb,n):
  
  m = matb.shape[0]
  k = matb.shape[1]

  vecb = matb.reshape(m//n, n, k).transpose((1, 0, 2)).reshape(n, m*k//n).transpose().reshape(m//n,n*k)

  #Return vectorised matrix
  return(vecb)

In [0]:
def comMat(a, b):

  # Example matrix with unique elements
  m = np.arange(a*b).reshape(a,b)

  # Vec(m) and Vec(m')
  vecm = mat2vec(m)
  vecmt = mat2vec(m.transpose())

  # Work out mapping between them.
  k=(vecm.transpose()==vecmt).astype(int).transpose()

  return(k)

## Lemmas

### **Lemma 1: Sum of $A_iB_i'$**


Let $A$ and $B$ be matrices of size $(m,k)$, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of size $(n,k)$ where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Then:

$$\sum_{i=1}^l A_iB_i' = ((A')^{r(kl,n)})'(B')^{r(kl,n)}$$

In [0]:
m = 8
n=2
l=m//n
k=4

print('Example Shapes:')
print('m: ', m)
print('k: ', k)
print('l: ', l)
print('n: ', n)

# A and B are random matrices
A = np.random.randn(m,k)
B = np.random.randn(m,k)

t1 = time.time()

# First we evaluate the LHS
sumAB = np.zeros((n,n))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumAB = sumAB + Ai @ Bi.transpose()

t2 = time.time()

# We print the time
time1=t2-t1
print('Time (LHS):')
print(time1)

# And the result
print('Result (LHS):')
print(sumAB)

# Now the RHS
t1 = time.time()
sumAB2 = A.transpose().reshape((m*k//n,n)).transpose() @ B.transpose().reshape((m*k//n,n))
t2 = time.time()

# We print the time
time2=t2-t1
print('Time (RHS):')
print(time2)

# And the result
print('Result (RHS):')
print(sumAB2)

# And the difference
print('Difference:')
print(sumAB-sumAB2)


# And the ratio between times
print('time ratio:')
print(time2/time1)

Example Shapes:
m:  8
k:  4
l:  4
n:  2
Time (LHS):
0.0005023479461669922
Result (LHS):
[[6.94992855 1.12747496]
 [9.38470056 3.32729359]]
Time (RHS):
9.799003601074219e-05
Result (RHS):
[[6.94992855 1.12747496]
 [9.38470056 3.32729359]]
Difference:
[[-8.88178420e-16  2.22044605e-16]
 [ 0.00000000e+00 -4.44089210e-16]]
time ratio:
0.1950640721404841


In [0]:
print(A)

print(A.transpose().reshape((m*k//n,n)).transpose())

[[-1.03758992 -0.9929396   2.25993744  1.45764701]
 [-0.55193597  1.42300798 -0.17094288  1.57698125]
 [ 2.22205484  0.21382045  0.44550954 -1.29899934]
 [ 1.40566434 -0.28190786  1.2002049  -2.1912631 ]
 [-0.66768399  1.16896045 -1.38153112 -0.7878413 ]
 [-1.63726715  1.76841371 -1.38325086 -1.63711714]
 [ 0.00706326  1.08959998 -0.74409399 -1.02894232]
 [-0.2313653  -0.90833872  1.7759244   1.07333532]]
[[-1.03758992  2.22205484 -0.66768399  0.00706326 -0.9929396   0.21382045
   1.16896045  1.08959998  2.25993744  0.44550954 -1.38153112 -0.74409399
   1.45764701 -1.29899934 -0.7878413  -1.02894232]
 [-0.55193597  1.40566434 -1.63726715 -0.2313653   1.42300798 -0.28190786
   1.76841371 -0.90833872 -0.17094288  1.2002049  -1.38325086  1.7759244
   1.57698125 -2.1912631  -1.63711714  1.07333532]]


### **Lemma 2: Sum of $A_iCB_i'$**


Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(m,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $C$ be a matrix of size $(k_1, k_2)$. Then:

$$\sum_{i=1}^l A_iCB_i' = ((A')^{r(k_1l,n)})'(CB')^{r(k_1l,n)} = ((C'A')^{r(k_2l,n)})'(B')^{r(k_2l,n)}$$

In [0]:
m = 1000
n=2
l=m//n
k1=100
k2=3

print('Example Shapes:')
print('m: ', m)
print('k1, k2: ', k1, k2)
print('l: ', l)
print('n: ', n)

# A, B and C are random matrices
A = np.random.randn(m,k1)
B = np.random.randn(m,k2)
C = np.random.randn(k1,k2)

t1 = time.time()

# First we evaluate the LHS
sumACB = np.zeros((n,n))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumACB = sumACB + Ai @ C @ Bi.transpose()

t2 = time.time()

# We print the time
time1=t2-t1
print('Time (LHS):')
print(time1)

# And the result
print('Result (LHS):')
print(sumACB)

# Now the middle
t1 = time.time()
sumACB2 = A.transpose().reshape((m*k1//n,n)).transpose() @ (C @ B.transpose()).reshape((m*k1//n,n))
t2 = time.time()

# We print the time
time2=t2-t1
print('Time (middle):')
print(time2)

# And the result
print('Result (middle):')
print(sumACB2)

# Now the RHS
t1 = time.time()
sumACB3 = (C.transpose() @ A.transpose()).reshape((m*k2//n,n)).transpose() @ B.transpose().reshape((m*k2//n,n))
t2 = time.time()

# We print the time
time3=t2-t1
print('Time (RHS):')
print(time3)

# And the result
print('Result (RHS):')
print(sumACB3)

# And the difference
print('Differences:')
print(sumACB-sumACB2)
print(sumACB2-sumACB3)
print(sumACB3-sumACB)


# And the ratio between times
print('time ratios:')
print(time2/time1)
print(time3/time2)
print(time1/time3)

Example Shapes:
m:  1000
k1, k2:  100 3
l:  500
n:  2
Time (LHS):
0.004046916961669922
Result (LHS):
[[  75.30258318 -686.5630488 ]
 [  16.42764312 -148.29578741]]
Time (middle):
0.002064943313598633
Result (middle):
[[  75.30258318 -686.5630488 ]
 [  16.42764312 -148.29578741]]
Time (RHS):
0.0009284019470214844
Result (RHS):
[[  75.30258318 -686.5630488 ]
 [  16.42764312 -148.29578741]]
Differences:
[[-1.98951966e-13 -2.27373675e-13]
 [ 1.42108547e-13 -1.98951966e-13]]
[[ 1.84741111e-13  2.27373675e-13]
 [-3.01980663e-13  1.42108547e-13]]
[[1.42108547e-14 0.00000000e+00]
 [1.59872116e-13 5.68434189e-14]]
time ratios:
0.5102509720749382
0.4496016626255629
4.359013867488444


### **Lemma 3: Sum of $(A_i-B_iC)(A_i-B_iC)'$**


Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(m,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $C$ be a matrix of size $(k_1, k_2)$. Then:

$$\sum_{i=1}^l (A_i-B_iC)(A_i-B_iC)'=((A')^{r(k_1l,n)})'(A')^{r(k_1l,n)}$$

### **Lemma 4: Sum of $A_i \otimes B_i$**

Let $A$ and $B$ be matrices of sizes $(m,k_1)$ and $(m,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{m/n}$ and $B_1,...B_{l}$ of sizes $(n,k_1)$ and $(n,k_2)$ respectively where $m|n$ and $l=m/n$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l} \end{bmatrix}, B = \begin{bmatrix} B_1 \\ B_2 \\ \vdots \\ B_{l} \end{bmatrix}$$

Let $a_i=vec(A_i)$ and $b_i=vec(B_i)$ for all $i \in \{1,...,l\}$ and define:

$$\tilde{a} = \begin{bmatrix} a_1 \\ a_2 \\ \vdots \\ a_{l} \end{bmatrix}, \tilde{b} = \begin{bmatrix} b_1 \\ b_2 \\ \vdots \\ b_{l} \end{bmatrix}$$

Then we have the following relationship:

$$\sum_{i=1}^{l} A_i \otimes B_i = ((I_{k_1} \otimes P_{n,k_2} \otimes I_{n})vec(\tilde{b}'\tilde{a}))^{r(n^2,k_1k_2)}$$

In [0]:
m = 4000
n=2
l=m//n
k1=4000
k2=1

print('Example Shapes:')
print('m: ', m)
print('k1, k2: ', k1, k2)
print('l: ', l)
print('n: ', n)

# A, B and C are random matrices
A = np.random.randn(m,k1)
B = np.random.randn(m,k2)

t1 = time.time()

# First we evaluate the LHS
sumAkB = np.zeros((n**2,k1*k2))
for i in np.arange(m//n):

  # Work out the row-wise chunks
  Ai = A[n*i:n*(i+1),:]
  Bi = B[n*i:n*(i+1),:]

  # Perform the summation
  sumAkB = sumAkB + np.kron(Ai, Bi)

t2 = time.time()
print('Time (LHS):')
print(t2-t1)
print('Shape (LHS):')
print(sumAkB.shape)

# Now for RHS

t1 = time.time()
# This matrix only needs be calculated once
K = comMat(n, k2)
IPI = np.kron(np.kron(np.eye(k1), K), np.eye(n))

t2 = time.time()
atilde = matb2vecb(A,n)
btilde = matb2vecb(B,n)

vecba = mat2vec(btilde.transpose() @ atilde)

RHS_noreshape = IPI @ vecba 

RHS_final = RHS_noreshape.reshape(k1*k2,n**2).transpose()


t3 = time.time()
print(t3-t1)
print(t3-t2)

print(np.allclose(RHS_final, sumAkB))
print(RHS_final)
print(sumAkB)




Example Shapes:
m:  4000
k1, k2:  4000 1
l:  2000
n:  2
Time (LHS):
4.968502044677734
Shape (LHS):
(4, 4000)
9.945883750915527
0.28938937187194824
True
[[  3.42441572  18.17508019  63.62925992 ...  55.98095479  28.59916152
  -32.34109229]
 [ -8.39253504 -42.67972678 -38.29225863 ...  -9.02363504  40.82507971
  -14.20268779]
 [ 63.37303062 -71.32816464 -61.47922823 ...  24.33618291 -90.81564465
  -43.89668294]
 [-72.79101746  -1.62471214   9.52570803 ... -46.98355562  48.78224011
  -49.16646219]]
[[  3.42441572  18.17508019  63.62925992 ...  55.98095479  28.59916152
  -32.34109229]
 [ -8.39253504 -42.67972678 -38.29225863 ...  -9.02363504  40.82507971
  -14.20268779]
 [ 63.37303062 -71.32816464 -61.47922823 ...  24.33618291 -90.81564465
  -43.89668294]
 [-72.79101746  -1.62471214   9.52570803 ... -46.98355562  48.78224011
  -49.16646219]]


### **Lemma 5: Double sum of $A_iB_j' \otimes C_iD_j'$**


Let $A$ and $C$ be matrices of sizes $(m_1,k_1)$ and $(m_1,k_2)$ respectively, partitioned row-wise into chunks $A_1,...A_{l_1}$ and $C_1,...C_{l_1}$ of sizes $(n_1,k_1)$ and $(n_1,k_2)$ respectively where $m_1|n_1$ and $l_1=m_1/n_1$. I.e.

$$A = \begin{bmatrix} A_1 \\ A_2 \\ \vdots \\ A_{l_1} \end{bmatrix}, C = \begin{bmatrix} C_1 \\ C_2 \\ \vdots \\ C_{l_1} \end{bmatrix}$$

Let $B$ and $D$ be matrices of sizes $(m_2,k_1)$ and $(m_2,k_2)$ respectively, partitioned row-wise into chunks $B_1,...B_{l_2}$ and $D_1,...D_{l_2}$ of sizes $(n_2,k_1)$ and $(n_2,k_2)$ respectively where $m_2|n_2$ and $l_2=m_2/n_2$. I.e.

$$B = \begin{bmatrix} B_1 \\ \vdots \\ B_{l_2} \end{bmatrix}, D = \begin{bmatrix} D_1\\ \vdots \\ D_{l_2} \end{bmatrix}$$

Then:

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_iB_j' \otimes C_iD_j' = ((I_{k_1} \otimes P_{n_1,k_2} \otimes I_{n_1})vec(\tilde{c}'\tilde{a}))^{r(n_1^2,k_1k_2)}\big[((I_{k_1} \otimes P_{n_2,k_2} \otimes I_{n_2})vec(\tilde{d}'\tilde{b}))^{r(n_2^2,k_1k_2)}\big]'$$

(This follows directly from lemma 4 and the mixed product property of the kronecker product).

In [0]:
# Dimensions
import scipy.sparse

# Partition sizes
n1 = 2
n2 = 2

# Number of rows
m1 = 1000
m2 = 1200

# Number of columns
k1 = 1000
k2 = 1000

# Matrices 

A = np.random.randn(m1,k1)
B = np.random.randn(m2,k1)
C = np.random.randn(m1,k2)
D = np.random.randn(m2,k2)

runningSum = np.zeros((n1**2,n2**2))
t1 = time.time()
# Work out LHS
for j in np.arange(m2//n2):

  Bj = B[n2*j:n2*(j+1),:]
  Dj = D[n2*j:n2*(j+1),:]

  for i in np.arange(m1//n1):

    Ai = A[n1*i:n1*(i+1),:]
    Ci = C[n1*i:n1*(i+1),:]

    runningSum = runningSum + np.kron(Ai @ Bj.transpose(), Ci @ Dj.transpose())

t2 = time.time()
print('Time (LHS)')
print(t2-t1)

# Now for RHS

t1 = time.time()
# This matrix only needs be calculated once
Kmat1 = scipy.sparse.csr_matrix(comMat(n1, k2))
IPI1 = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(k1), Kmat1), scipy.sparse.identity(n1))
Kmat2 = scipy.sparse.csr_matrix(comMat(n2, k2))
IPI2 = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(k1), Kmat2), scipy.sparse.identity(n2))

t2 = time.time()
atilde = matb2vecb(A,n1)
btilde = matb2vecb(B,n2)
ctilde = matb2vecb(C,n1)
dtilde = matb2vecb(D,n2)

vecca = mat2vec(ctilde.transpose() @ atilde)
vecdb = mat2vec(dtilde.transpose() @ btilde)

RHS1_noreshape = IPI1 @ vecca 
RHS2_noreshape = IPI2 @ vecdb 

RHS1_final = RHS1_noreshape.reshape(k1*k2,n1**2).transpose()
RHS2_final = RHS2_noreshape.reshape(k1*k2,n2**2)

RHS_final = RHS1_final @ RHS2_final

t3 = time.time()
print('Time (preprocessing)')
print(t2-t1)
print('Time (RHS)')
print(t3-t2)

print(np.allclose(RHS_final, runningSum))


Time (LHS)
11.228847980499268
Time (preprocessing)
0.36844754219055176
Time (RHS)
0.45260167121887207
True


### **Lemma 6: Double sum of $A_{i,j} \otimes B_iC_j'$**


Let $A$ be a matrix of size $(m_1,m_2)$, partitioned block-wise into chunks $\{A_{ij}\}_{i=1,...l_1}^{j=1...l_2}$ of size $(n_1,n_2)$ where $m_1|n_1$ with $l_1=m_1/n_1$ $m_2|n_2$ with $l_2=m_2/n_2$. I.e.

$$A = \begin{bmatrix} A_{11} & A_{12} & ... & A_{1l_2} \\ A_{21} & A_{22} & ... & A_{2l_2} \\ \vdots \\ A_{l_11} & A_{l_12} & ... & A_{l_1l_2} \end{bmatrix}$$

Let $B$ and $C$ be matrices of sizes $(m_1,k)$ and $(m_2,k)$ respectively, partitioned row-wise into chunks $B_1,...B_{l_1}$ and $C_1,...C_{l_2}$ of sizes $(n_1,k)$ and $(n_2,k)$ respectively. I.e.

$$B = \begin{bmatrix} B_1 \\ \vdots \\ B_{l_1} \end{bmatrix}, C = \begin{bmatrix} C_1 \\ C_2 \\ \vdots \\ C_{l_2} \end{bmatrix}$$

Then:

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{ij} \otimes B_iC_j' = ((I_{n_2} \otimes P_{n_1,n_2} \otimes I_{n_1})vec(\widetilde{(bc')}'\tilde{a}))^{r(n_1^2,n_2^2)}$$

$$\widetilde{(bc')}=vecb(BC')$$
$$\widetilde{a}=vecb(A)$$

In [0]:
n1 = 2
n2 = 2
m1 = 400
m2 = 800
k=100
l1 = m1//n1
l2 = m2//n2

# Create A
A = np.random.randn(m1,m2)

B = np.random.randn(m1,k)
C = np.random.randn(m2,k)

runningSum = np.zeros((n1**2,n2**2))

# LHS calculation
t1 = time.time()
for i in np.arange(l1):
  for j in np.arange(l2):

    Aij = A[n1*i:n1*(i+1),n2*j:n2*(j+1)]
    Bi = B[n1*i:n1*(i+1),:]
    Cj = C[n2*j:n2*(j+1),:]

    runningSum = runningSum + np.kron(Aij, Bi @ Cj.transpose())

t2 = time.time()
print(t2-t1)

# RHS calculation
t1 = time.time()

# Change A to stacked form
As = A.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)

# B times C transpose
BCt = B @ C.transpose()

# B times C transpose (stacked)
BCts = BCt.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)

# This matrix only needs be calculated once
K = comMat(n1, n2)
IPI = scipy.sparse.kron(scipy.sparse.kron(scipy.sparse.identity(n2), K), scipy.sparse.identity(n1))

t2 = time.time()
atilde = matb2vecb(As,n1)
bctilde = matb2vecb(BCts,n1)

vecbca = mat2vec(bctilde.transpose() @ atilde)

RHS_noreshape = IPI @ vecbca 

RHS_final = RHS_noreshape.reshape(n1**2,n2**2).transpose()


t3 = time.time()
print(t3-t1)
print(t3-t2)

print(np.allclose(RHS_final, runningSum))
print(RHS_final)
print(runningSum)

2.628631353378296
0.020653724670410156
0.009497404098510742
True
[[ 1550.21361498  2590.53767701 -2047.52861912   213.8205423 ]
 [  268.72422262   977.72926385   -52.94880813  1720.64550173]
 [  742.71205499  3316.38039641  4157.54250268  2200.37850575]
 [ -886.16548126   950.98653872  5493.9735444   3485.59525685]]
[[ 1550.21361498  2590.53767701 -2047.52861912   213.8205423 ]
 [  268.72422262   977.72926385   -52.94880813  1720.64550173]
 [  742.71205499  3316.38039641  4157.54250268  2200.37850575]
 [ -886.16548126   950.98653872  5493.9735444   3485.59525685]]


## Comments

This section contains comments relating the lemmas above to BLMM

### Derivative of $l$ with respect to $D_k$

We already have that; $$\frac{\partial l}{\partial D_k} = \frac{1}{2}\sum_{j=1}^{l_k}(T_{(k,j)}u)(T_{(k,j)}u)' - \frac{1}{2}\sum_{j=1}^kT_{(k,j)}T_{(k,j)}'$$


Consider the first term, given by:
$$\frac{1}{2}\sum_{j=1}^{l_k}(T_{(k,j)}u)(T_{(k,j)}u)'$$
$$= \sigma^{-2}\frac{1}{2}\sum_{j=1}^{l_k}(Z'_{(k,j)}e-Z'_{(k,j)}ZD(I+Z'ZD)^{-1}Z'e)(e'Z_{(k,j)}-e'ZD(I+Z'ZD)^{-1}Z'Z_{(k,j)})$$
$$ = \sigma^{-2}\frac{1}{2}\sum_{j=1}^{l_k} (A_j - B_j)(A_j-B_j)'$$

Upon letting:
 - $A_j = Z'_{(k,j)}e$ with $A = Z'_{(k)}e$ 
 - $B_j = Z'_{(k,j)}ZD(I+Z'ZD)^{-1}Z'e$ with $B = Z'_{(k)}ZD(I+Z'ZD)^{-1}Z'e$ 

Noting, now, by Lemma 1, that this is equal to:

$$= \sigma^{-2}\frac{1}{2}((A')^{r(l_k,q_k)}-(B')^{r(l_k,q_k)})'((A')^{r(l_k,q_k)}-(B')^{r(l_k,q_k)})$$

$$= \sigma^{-2}\frac{1}{2}\bigg((e'Z_{(k)})^{r(l_k,q_k)}-(e'Z(I+DZ'Z)^{-1}DZ'Z_{(k)})^{r(l_k,q_k)}\bigg)'\bigg((e'Z_{(k)})^{r(l_k,q_k)}-(e'Z(I+DZ'Z)^{-1}DZ'Z_{(k)})^{r(l_k,q_k)}\bigg)$$

This is how the first term is computed in a computationally efficient manner. The second term can be computed by first noting:

$$\frac{1}{2}\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'=\frac{1}{2}\sum_{j=1}^{l_k}\bigg(Z_{(k,j)}'Z_{(k,j)} - Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z_{(k,j)}\bigg)$$
$$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \frac{1}{2}\sum_{j=1}^{l_k}\bigg( Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z_{(k,j)}\bigg)$$
$$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \frac{1}{2}\sum_{j=1}^{l_k}A_jB_j'$$

Upon letting:
 - $A_j = Z'_{(k,j)}Z$ with $A = Z'_{(k)}Z$ 
 - $B_j = Z'_{(k,j)}Z(I+DZ'Z)^{-1}D$ with $B = Z'_{(k)}Z(I+DZ'Z)^{-1}D$ 

Upon applying lemma 1, we see that this equals;

 $$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((A')^{r(l_k\sum_il_iq_i,q_k)})'(B')^{r(l_k\sum_il_iq_i,q_k)}$$
 $$=\frac{1}{2}\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((Z'Z_{(k,j)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z_{(k,j)})^{r(l_k\sum_il_iq_i,q_k)}$$

Note that the first term can be computed prior to any iteration as it does not depend on any parameter estimates and therefore can be ignored here.

### Covariance of derivative of $l$ with respect to $D_k$ and $l$ with respect to $\sigma^2$ 


The covariance of the $\frac{\partial l}{\partial \sigma^2}$ and $\frac{\partial l}{\partial D_k}$ is given by:

$$cov\bigg(\frac{\partial l}{\partial \sigma^2},\frac{\partial l}{\partial D_k}\bigg) = \frac{1}{2\sigma^2}\text{vec}'\bigg(\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'\bigg)\mathcal{D^{+'}}$$




The second term can be computed by first noting:

$$\sum_{j=1}^{l_k}T_{(k,j)}T_{(k,j)}'=\sum_{j=1}^{l_k}\bigg(Z_{(k,j)}'Z_{(k,j)} - Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z_{(k,j)}\bigg)$$
$$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \sum_{j=1}^{l_k}\bigg( Z_{(k,j)}'ZD(I+Z'ZD)^{-1}Z_{(k,j)}\bigg)$$
$$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - \sum_{j=1}^{l_k}A_jB_j'$$

Upon letting:
 - $A_j = Z'_{(k,j)}Z$ with $A = Z'_{(k)}Z$ 
 - $B_j = Z'_{(k,j)}Z(I+DZ'Z)^{-1}D$ with $B = Z'_{(k)}Z(I+DZ'Z)^{-1}D$ 

Upon applying lemma 1, we see that this equals;

 $$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((A')^{r(l_k\sum_il_iq_i,q_k)})'(B')^{r(l_k\sum_il_iq_i,q_k)}$$
 $$=\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)} - ((Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)}$$

Note that the first term can be computed prior to any iteration as it does not depend on any parameter estimates and therefore can be ignored here. Therefore, we have:

$$cov\bigg(\frac{\partial l}{\partial \sigma^2},\frac{\partial l}{\partial D_k}\bigg) = \frac{1}{2\sigma^2}\text{vec}'\bigg(\sum_{j=1}^{l_k}Z_{(k,j)}'Z_{(k,j)}\bigg)\mathcal{D^{+'}} - \frac{1}{2\sigma^2}\text{vec}'\bigg(((Z'Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)})'(D(I+Z'ZD)^{-1}Z_{(k)})^{r(l_k\sum_il_iq_i,q_k)}\bigg)\mathcal{D^{+'}}$$

### Covariance of derivative of $l$ with respect to $D_k$

The covariance of the $\frac{\partial l}{\partial D_{k_1}}$ and $\frac{\partial l}{\partial D_{k_2}}$ is given by:

$$\text{cov}\bigg(\frac{\delta l}{\delta \text{vech}(D_{k_1})},\frac{\delta l}{\delta \text{vech}(D_{k_2})}\bigg)=\frac{1}{2}\mathcal{D}_{k_1}^+\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(R_{(k_1,k_2,i,j)}\otimes R_{(k_1,k_2, i,j)}
)\mathcal{D}_{k_2}^{+'}$$

Where $R_{(k_1,k_2,i,j)}=Z'_{(k_1,i)}Z_{(k_2,j)}-Z_{(k_1,i)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k_2,j)}$. Let:

 - $A_{i,j}=Z'_{(k_1,i)}Z_{(k_2,j)}$ where $A=Z'Z_{(k_1,k_2)}$.
 - $B_{i,j}=Z_{(k_1,i)}'ZD(I+Z'ZD)^{-1}Z'Z_{(k_2,j)}$

It follows that:

 $$\text{cov}\bigg(\frac{\delta l}{\delta \text{vech}(D_{k_1})},\frac{\delta l}{\delta \text{vech}(D_{k_2})}\bigg)=\frac{1}{2}\mathcal{D}_{k_1}^+\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}\bigg((A_{i,j} - B_{i,j})\otimes (A_{i,j} - B_{i,j})\bigg)\mathcal{D}_{k_2}^{+'}$$
 $$=\frac{1}{2}\mathcal{D}_{k_1}^+\bigg(\sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(A_{i,j}\otimes A_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(A_{i,j}\otimes B_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(B_{i,j}\otimes A_{i,j}) - \sum_{j=1}^{l_{k_2}}\sum_{i=1}^{l_{k_1}}(B_{i,j}\otimes B_{i,j})\bigg)\mathcal{D}_{k_2}^{+'}$$

Each of these can be calculated in the below mannor via a variant of lemma 6:


$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{i,j} \otimes A_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{a}'\tilde{a}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}B_{i,j} \otimes A_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{b}'\tilde{a}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}A_{i,j} \otimes B_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{a}'\tilde{b}))^{r(q_1^2,q_2^2)}$$

$$\sum_{j=1}^{l_2} \sum_{i=1}^{l_1}B_{i,j} \otimes B_{i,j} = ((I_{q_2} \otimes P_{q_1,q_2} \otimes I_{q_1})vec(\tilde{b}'\tilde{b}))^{r(q_1^2,q_2^2)}$$

Where the lower case letter and tilde notation represents the $vecb(\cdot)$ operator. I.e. if $X$ is partitioned into blocks of size $(n_1 \times n_2)$ like so:

$$X = \begin{bmatrix} X_{1,1} & X_{1,2} & ... & X_{1,l_2} \\ X_{2,1} & X_{2,2} & ... & X_{2,l_2} \\ \vdots \\ X_{l_1,1} & X_{l_1,2} & ... & X_{l_1,l_2} \end{bmatrix}$$

Then $\tilde{x}=vecb^{(n_1,n_2)}(X)$ is given by:

$$\tilde{x} = \begin{bmatrix} vec'(X_{1,1}) \\ vec'(X_{1,2}) \\ \vdots \\ vec'(X_{1,l_2}) \\ vec'(X_{2,1}) \\ \vdots \\ vec'(X_{l_1,l_2}) \end{bmatrix}$$

It is also worth noting that the first sum given above does not involve any estimated parameters so does not need to be recomputed for each iteration and can be computed beforehand.

## Final Functions

This section contains functions, and tests of those functions, intended for use in BLMM, based on the previous lemmas.

### Function: ``sumAijBijt``

This function takes in the following inputs:

 - A: A 2D matrix of dimension $(m_1 \times m_2)$.
 - B: A 2D matrix of dimension $(m_3 \times m_4)$.
 - pA: The size of the block partitions of $A$, e.g. if $A_{i,j}$ is of dimension $(n_1 \times n_2)$ the ``pA=[n1, n2]``.
 - pB: The size of the block partitions of $B$, e.g. if $B_{i,j}$ is of dimension $(n_3 \times n_4)$ the ``pB=[n3, n4]``.

And gives the following output:

 - S: The sum of the partitions of $A$ multiplied by the transpose of the partitions of $B$; i.e. 
 
 $$\sum_{j=1}^{l_1}\sum_{i=1}^{l_2} A_{i,j}B_{i,j}'$$
  
  where $l_1:=\frac{m_1}{n_1}=\frac{m_3}{n_3}$ and $l_2:=\frac{m_2}{n_2}=\frac{m_4}{n_4}$.

In [0]:
n1 = 2
n2 = 2
m1 = 400
m2 = 800
k=100
l1 = m1//n1
l2 = m2//n2

# Create A
A = np.random.randn(m1,m2)

B = np.random.randn(m1,k)
C = np.random.randn(m2,k)


# RHS calculation
t1 = time.time()

# Change A to stacked form
As = A.reshape((m1//n1,n1,m2//n2,n2)).transpose(0,2,1,3).reshape(m1*m2//n2,n2)